## step 1: read in data and extract pos/neg reviews

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd

In [0]:
data_toy = pd.read_csv('/content/drive/My Drive/american_traditional_review.csv',nrows=1000)

In [0]:
data_pos = data_toy['text'][data_toy['stars'] >= 4.0]
data_neg = data_toy['text'][data_toy['stars'] <= 2.0]

In [0]:
corpus_pos = list(data_pos)
corpus_neg = list(data_neg)

## step 2: data preprocessing

*   ### tokenization
*   ### stopwords
*   ### lemmatized
*   ### stemmed

In [0]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [25]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

### lemmatize eg

In [27]:
print(WordNetLemmatizer().lemmatize('bought', pos = 'v')) # past tense to present tense

buy


### stemmer eg

In [30]:
import pandas as pd
stemmer = SnowballStemmer("english")
original_words = ['caresses', 'flies', 'dies', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]

pd.DataFrame(data={'original word':original_words, 'stemmed':singles })

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,itemization,item
4,sensational,sensat
5,traditional,tradit
6,reference,refer
7,colonizer,colon
8,plotted,plot


### wrap all transformation into a function

In [0]:
def preprocess(text):
  result = []
  for token in gensim.utils.simple_preprocess(text):
    if token not in gensim.parsing.preprocessing.STOPWORDS:
      token = stemmer.stem(WordNetLemmatizer().lemmatize(token, pos='v'))
      result.append(token)
  return result

In [33]:
doc_sample = "I was told so many great things about this place but my entire experience was awful."

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['I', 'was', 'told', 'so', 'many', 'great', 'things', 'about', 'this', 'place', 'but', 'my', 'entire', 'experience', 'was', 'awful.']


Tokenized and lemmatized document: 
['tell', 'great', 'thing', 'place', 'entir', 'experi', 'aw']


In [0]:
processed_pos = []
for doc in corpus_pos:
    processed_pos.append(preprocess(doc))
    
processed_neg = []
for doc in corpus_neg:
    processed_neg.append(preprocess(doc))
    

## step 3: bags of words

In [45]:
dict_pos = gensim.corpora.Dictionary(processed_pos)

count = 0
for k, v in dict_pos.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 avoid
1 bar
2 belief
3 best
4 breakfast
5 buffet
6 chewi
7 chimichurri
8 chocol
9 cold
10 cooki


In [0]:
bow_corpus = [dict_pos.doc2bow(doc) for doc in processed_pos]

In [50]:
document_num = 50
bow_doc_x = bow_corpus[document_num]

for i in range(len(bow_doc_x)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_x[i][0], 
                                                     dict_pos[bow_doc_x[i][0]], 
                                                     bow_doc_x[i][1]))

Word 4 ("breakfast") appears 1 time.
Word 27 ("good") appears 2 time.
Word 77 ("servic") appears 1 time.
Word 186 ("kid") appears 1 time.
Word 233 ("awesom") appears 1 time.
Word 234 ("benedict") appears 2 time.
Word 249 ("price") appears 1 time.
Word 269 ("love") appears 1 time.
Word 293 ("friend") appears 1 time.
Word 640 ("wow") appears 1 time.
Word 791 ("approv") appears 1 time.
Word 792 ("fact") appears 1 time.
Word 793 ("florentin") appears 1 time.
Word 794 ("mom") appears 1 time.
Word 795 ("rubin") appears 1 time.


## running LDA using bow

In [0]:
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 5, 
                                   id2word = dict_pos,                                    
                                   passes = 10,
                                   workers = 2)

In [56]:
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.021*"good" + 0.018*"food" + 0.017*"great" + 0.011*"servic" + 0.010*"place" + 0.009*"come" + 0.009*"like" + 0.007*"best" + 0.006*"get" + 0.006*"delici"


Topic: 1 
Words: 0.010*"place" + 0.009*"like" + 0.009*"food" + 0.009*"come" + 0.009*"time" + 0.008*"good" + 0.007*"ve" + 0.007*"love" + 0.007*"best" + 0.006*"tri"


Topic: 2 
Words: 0.020*"great" + 0.020*"food" + 0.016*"place" + 0.013*"good" + 0.012*"time" + 0.011*"servic" + 0.011*"breakfast" + 0.009*"love" + 0.009*"come" + 0.009*"like"


Topic: 3 
Words: 0.028*"place" + 0.013*"great" + 0.012*"food" + 0.011*"good" + 0.009*"love" + 0.007*"price" + 0.007*"like" + 0.007*"time" + 0.006*"get" + 0.005*"eat"


Topic: 4 
Words: 0.014*"food" + 0.012*"order" + 0.011*"great" + 0.010*"friend" + 0.010*"drink" + 0.010*"dog" + 0.010*"good" + 0.009*"servic" + 0.009*"come" + 0.007*"restaur"


